# **Bài tập lớn cuối kỳ**
- Mssv: 1712601
- Họ và tên: Trịnh Văn Minh

# Ý tưởng để tái tạo ảnh mật



- Nhân ma trận nghịch đảo để giải hệ phương trình 
- Từ đó tìm ra các pixel $P_i$
- Hệ có dạng

\begin{equation*}
\begin{bmatrix}
P_1x_1^0 & P_2x_1^2 & P_3x_1^2 \\
P_1x_2^0 & P_2x_2^2 & P_3x_2^2 \\
P_1x_3^0 & P_2x_3^2 & P_3x_3^2
\end{bmatrix}
=
\begin{bmatrix}
y_1 \\
y_2 \\
y_3
\end{bmatrix}
\end{equation*}

- Tương đương

\begin{equation*}
\begin{bmatrix}
x_1^0 & x_1^2 & x_1^2 \\
x_2^0 & x_2^2 & x_2^2 \\
x_3^0 & x_3^2 & x_3^2
\end{bmatrix}
\begin{bmatrix}
P_1 \\
P_2 \\
P_3
\end{bmatrix}
=
\begin{bmatrix}
y_1 \\
y_2 \\
y_3
\end{bmatrix}
\end{equation*}

- Áp dụng phương pháp nhân nghịch đảo để tính các pixel ($P_i$)

\begin{equation*}
A 
= 
\begin{bmatrix}
x_1^0 & x_1^2 & x_1^2 \\
x_2^0 & x_2^2 & x_2^2 \\
x_3^0 & x_3^2 & x_3^2
\end{bmatrix}
\end{equation*}

\begin{equation*}
X 
= 
\begin{bmatrix}
P_1 \\
P_2 \\
P_3
\end{bmatrix}
\end{equation*}

\begin{equation*}
B 
= 
\begin{bmatrix}
y_1 \\
y_2 \\
y_3
\end{bmatrix}
\end{equation*}

$A.X = B $

$X^T.A^T$ = $B^T$

$X^T$ = $B^T.(A^T)^(-1)$

# Phần code


In [124]:
import math
from PIL import Image
import numpy as np
from sympy import Matrix

In [125]:
def compute_polynomial(p, coefs, xs):
    '''
    Tính giá trị của hàm đa thức.

    Các tham số:
        coefs (mảng numpy một chiều): Các hệ số của hàm đa thức, 
                                        f(x) = coefs[0] + coefs[1]*x + coefs[2]*x^2 + ...
        xs (mảng numpy một chiều): Các giá trị x mà tại đó cần tính f(x).
    Giá trị trả về:
        Mảng numpy một chiều: Phần tử chỉ số i trong mảng này bằng f(xs[i]).
    '''
    # TODO
    f_xs = np.zeros(xs.shape, dtype=int)
    for i in range(len(coefs)):
        xs_pow_i = np.ones(xs.shape, dtype=int)
        for j in range(i):
            xs_pow_i = (xs_pow_i * xs) % p

        f_xs = (f_xs + coefs[i] * xs_pow_i % p) % p

    return f_xs

def convert_pixels_to_mode(pixels, mode=0):
  # lossy
  if mode == 0:
      for r in range(pixels.shape[0]):
          for c in range(pixels.shape[1]):
              if pixels[r,c] > 250:
                  pixels[r,c] = 250
      return pixels
  # mode 1 or any: lossless
  else:
      pixels = pixels.flatten()
      new_pixels = np.array([], dtype=np.uint8)

      for i in range(pixels.size):
        if pixels[i] >= 250:
          new_pixels = np.append(new_pixels, [250, pixels[i] - 250])
        else:
          new_pixels = np.append(new_pixels, pixels[i])
      
      return new_pixels


def split(ori_image, split_image, n, k, p=251, mode=0):
    '''
    Hàm phân chia ảnh mật ori_image m pixel thành k phần (k nhóm) sao cho chỉ khi có ít nhất là k phần (k <= n)
    thì mới tái tạo được ori_image, còn ít hơn k phần thì sẽ không biết gì về s.

    Các tham số:
        ori_image (*.bmp): Ảnh cần chia sẻ.
        split_image (*.bmp): Phần tên không bao gồm số thứ tự (vd part.bmp thay vì part0.bmp)
        n (int): Số phần cần phân chia.
        k (int): Ngưỡng k (k <= n).
        mode: 0 (lossy)
              1 (lossless)
    Giá trị trả về:
        n ảnh mang tên split_image$.bmp
        xs mảng x
    '''
    pixels = np.array(Image.open(ori_image))

    ori_shape = pixels.shape
    m = pixels.size
    
    # convert pixels to mode
    pixels = convert_pixels_to_mode(pixels, mode)
    new_m = pixels.size

    # padding 0 -> m % k == 0
    padded_pixels = np.append(pixels, (k - new_m % k)*[0])
    # reshape into group
    padded_pixels = padded_pixels.reshape(-1, k)
    groups_len = padded_pixels.shape[0]
    # 2D array
    result_arr = np.zeros((groups_len, n), dtype=np.uint8)

    for i in range(groups_len):
        # Lấy các hệ số từ k pixels
        coefs = padded_pixels[i]
        xs = np.arange(n) + 1
        ys = compute_polynomial(p, coefs, xs)
        ys = ys.astype(np.uint8)

        result_arr[i] = ys

    # Tách từng phần, tách từng ảnh
    for i in range(n):
        new_pixels = result_arr[:, i]
        # padding
        new_pixels = np.append(new_pixels, [0]*(m - len(new_pixels)))
        new_pixels = new_pixels.reshape(ori_shape[0], ori_shape[1])
        new_pixels= new_pixels.astype(np.uint8)
        
        # Ghi new pixels xuống file
        Image.fromarray(new_pixels).save(split_image.split('.')[0] + str(i + 1) + '.bmp')
    return xs

## Hàm get_ys_group 
- Với tham số đầu vào split_image (vd: part.bmp thay vì part1.bmp)
- Hàm này sẽ trả về các nhóm các ys, kích thước (512*512), shape(512,512)

In [126]:
def get_ys_group(split_image, n, k, p=251):
    pixels_part = np.array(Image.open(split_image.split('.')[0] + '1.bmp'))
    m = pixels_part.size
    shape = pixels_part.shape
    # kích thước ban đầu của part (chưa padding)
    ori_size = math.ceil(m / k)
    ys_group = np.zeros((n, ori_size), dtype=int)
    for i in range(n):
        pixels_part = np.array(Image.open(
            split_image.split('.')[0] + str(i + 1) + '.bmp'))
        # loại bỏ padding
        pixels_part = pixels_part.flatten()[:ori_size]
        ys_group[i] = pixels_part
    ys_group = ys_group.T
    return ys_group, m, shape

## Hàm get_A 
- Nhận vào tham số `new_xs` là một mảng bao gồm các xs đã được hoán vị ngẫu nhiên
- Tạo ra mảng `xs_pow_i_arr` là một matrix gôm các giá trị
\begin{equation*}
A 
= 
\begin{bmatrix}
x_1^0 & x_1^2 & x_1^2 \\
x_2^0 & x_2^2 & x_2^2 \\
x_3^0 & x_3^2 & x_3^2
\end{bmatrix}
\end{equation*}

- Trà về một matrix A đã nghịch đảo và mod 251



In [127]:
def get_A(new_xs, k, p=251):
    # print(new_xs)
    xs_pow_i_arr = np.zeros((k, k), dtype=np.uint8)
    for i in range(len(new_xs)):
        xs_pow_i = np.ones(new_xs.shape, dtype=np.uint8)
        for j in range(i):
            xs_pow_i = (xs_pow_i * new_xs) % 251
        xs_pow_i_arr[i] = xs_pow_i
    # print(xs_pow_i_arr)

    # inverse matrix mod p
    A = Matrix(xs_pow_i_arr)  # keyMatrix is your basic matrix ndrarray format
    A = A.inv_mod(p)  # 251
    return A

## Hàm join
- Nhận vào các tham số đã được cắt ra bởi các index ngẫu nhiên của xs
- Trả về kết quả là một mảng gồm các pixel được tính bằng

 `pixels_part = matrix_B * matrix_A % p`

In [135]:
def join(new_xs, new_ys_group, k, p=251):
    '''
    Tái tạo tin mật s từ n' phần của tin mật (n' >= k).

    Các tham số:
        xs, ys (2 mảng numpy một chiều, len = n'): n' phần của tin mật: phần thứ 1 là (xs[0], ys[0]), 
                                                                        phần thứ 2 là (xs[1], ys[1]), 
                                                                        ...
        k (int): Ngưỡng k mà đã dùng khi phân chia tin mật.
    Giá trị trả về:
        int: Tin mật s được tái tạo,
                trong trường hợp không đủ số phần để tái tạo tin mật thì trả về None.
    '''
    # TODO
    if len(new_xs) < k:
        return None

    # A inversed, mod 251
    matrix_A = get_A(new_xs, k, p)

    pixels = np.array([], dtype=int)
    for i in range(new_ys_group.shape[0]):
        matrix_B = Matrix(new_ys_group[i]).reshape(1, k)  # 1, k
        pixels_part = matrix_B * matrix_A % p
        pixels = np.append(pixels, pixels_part)
  
    return pixels.astype(np.uint8)

# Phân chia ảnh mật: chia nhóm, lấy kích thước ban đầu, shape

In [129]:
# Phân chia ảnh mật
p = 251
k = 3
n = 5

# mode: 0 (lossy), 1(lossless)
xs = split('cover.bmp','part.bmp', n, k, p, mode=0)

# chia nhóm
ys_group, size, shape = get_ys_group('part.bmp', n, k, p)
print(ys_group)
print(xs)
print(size, shape)

[[235 130  98 139   2]
 [229 109  52  58 127]
 [239 145 132 200  98]
 ...
 [ 40 198  62 134 163]
 [ 58 226 102 188 233]
 [108 108 108 108 108]]
[1 2 3 4 5]
262144 (512, 512)


# Chọn ngẫu nhiên n' phần để tái tạo tin mật 
Mất khoảng 2 phút với n = 5, k = 3

In [136]:
# -------------------------------------
#   mất khoảng 2 phút để hoàn tất
#--------------------------------------

rand_idxs = np.random.permutation(np.arange(n))

# -------------------------------------
n_prime = k - 1

new_xs = xs[rand_idxs[:n_prime]]
new_ys_group = ys_group[:, rand_idxs[:n_prime]]

rec_s = join(new_xs, new_ys_group, k, p)
print(rec_s)

#--------------------------------------

n_prime = k

new_xs = xs[rand_idxs[:n_prime]]
new_ys_group = ys_group[:, rand_idxs[:n_prime]]

rec_s = join(new_xs, new_ys_group, k, p)
rec_s = rec_s[:size].reshape(shape)
print(rec_s)

# Ghi new pixels xuống file
Image.fromarray(rec_s).save('reconstructor_cover.bmp')

None
[[162 162 162 ... 170 155 128]
 [162 162 162 ... 170 155 128]
 [162 162 162 ... 170 155 128]
 ...
 [ 43  43  50 ... 104 100  98]
 [ 44  44  55 ... 104 105 108]
 [ 44  44  55 ... 104 105 108]]


# Kiểm tra lại kết quả

In [137]:

pixels = np.array(Image.open('cover.bmp'))
print(pixels)
print(pixels.size)
print(pixels.shape)
print(pixels.dtype)

print(rec_s)
print(rec_s.size)
print(rec_s.shape)
print(rec_s.dtype)

number_of_equal_elements = np.sum(pixels==rec_s)
total_elements = np.multiply(*pixels.shape)
percentage = number_of_equal_elements/total_elements

print('total number of elements: \t\t{}'.format(total_elements))
print('number of identical elements: \t\t{}'.format(number_of_equal_elements))
print('number of different elements: \t\t{}'.format(total_elements-number_of_equal_elements))
print('percentage of identical elements: \t{:.2f}%'.format(percentage*100))



[[162 162 162 ... 170 155 128]
 [162 162 162 ... 170 155 128]
 [162 162 162 ... 170 155 128]
 ...
 [ 43  43  50 ... 104 100  98]
 [ 44  44  55 ... 104 105 108]
 [ 44  44  55 ... 104 105 108]]
262144
(512, 512)
uint8
[[162 162 162 ... 170 155 128]
 [162 162 162 ... 170 155 128]
 [162 162 162 ... 170 155 128]
 ...
 [ 43  43  50 ... 104 100  98]
 [ 44  44  55 ... 104 105 108]
 [ 44  44  55 ... 104 105 108]]
262144
(512, 512)
uint8
total number of elements: 		262144
number of identical elements: 		262144
number of different elements: 		0
percentage of identical elements: 	100.00%


# Phân chia tin mật 2

In [34]:

p = 251
k = 25
n = 50
xs = split('cover.bmp','k25_part.bmp', n, k, p)
# chia nhóm
ys_group, size, shape = get_ys_group('k25_part.bmp', n, k, p)
print(ys_group)
print(xs)
print(size, shape)

[[217 143 162 ... 235  73  64]
 [ 47  23 228 ... 149  47  41]
 [123  10 118 ...  21 225 133]
 ...
 [ 94 101 108 ...  91 124 237]
 [115  22 212 ...  74 112 158]
 [190  32  65 ... 172 181 170]]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50]
262144 (512, 512)


# Chọn ngẫu nhiên n' phần để tái tạo tin mật
Mất khoảng 11 phút để tái tạo với k = 25 và n = 50

In [35]:
# -------------------------------------
#   mất khoảng 11 phút để hoàn tất
#--------------------------------------

rand_idxs = np.random.permutation(np.arange(len(xs)))

n_prime = k

new_xs = xs[rand_idxs[:n_prime]]
new_ys_group = ys_group[:, rand_idxs[:n_prime]]

rec_s = join(new_xs, new_ys_group, k, p)
rec_s = rec_s[:size].reshape(shape)
print(rec_s)

# Ghi new pixels xuống file
Image.fromarray(rec_s).save('k25_reconstructor_cover.bmp')

[[162 162 162 ... 170 155 128]
 [162 162 162 ... 170 155 128]
 [162 162 162 ... 170 155 128]
 ...
 [ 43  43  50 ... 104 100  98]
 [ 44  44  55 ... 104 105 108]
 [ 44  44  55 ... 104 105 108]]


# Kiểm tra lại kết quả

In [39]:

pixels = np.array(Image.open('cover.bmp'))
print(pixels)
print(pixels.size)
print(pixels.shape)
print(pixels.dtype)

print(rec_s)
print(rec_s.size)
print(rec_s.shape)
print(rec_s.dtype)


number_of_equal_elements = np.sum(pixels==rec_s)
total_elements = np.multiply(*pixels.shape)
percentage = number_of_equal_elements/total_elements

print('total number of elements: \t\t{}'.format(total_elements))
print('number of identical elements: \t\t{}'.format(number_of_equal_elements))
print('number of different elements: \t\t{}'.format(total_elements-number_of_equal_elements))
print('percentage of identical elements: \t{:.2f}%'.format(percentage*100))

[[162 162 162 ... 170 155 128]
 [162 162 162 ... 170 155 128]
 [162 162 162 ... 170 155 128]
 ...
 [ 43  43  50 ... 104 100  98]
 [ 44  44  55 ... 104 105 108]
 [ 44  44  55 ... 104 105 108]]
262144
(512, 512)
uint8
[[162 162 162 ... 170 155 128]
 [162 162 162 ... 170 155 128]
 [162 162 162 ... 170 155 128]
 ...
 [ 43  43  50 ... 104 100  98]
 [ 44  44  55 ... 104 105 108]
 [ 44  44  55 ... 104 105 108]]
262144
(512, 512)
uint8
total number of elements: 		262144
number of identical elements: 		262144
number of different elements: 		0
percentage of identical elements: 	100.00%
